# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine

In [2]:
# Create connection to database
engine = create_engine('sqlite:///message_features.db')
con = engine.connect()

# Find the names of tables inside of this database
engine.table_names()

['messages']

In [3]:
# Read in data to pandas dataframe
df = pd.read_sql_table("messages", con)

df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Create X and Y dataframes
X = df["message"]
Y = df.drop(labels=["message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # Normalise by setting to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Create tokens 
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # Lemmatise words
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
basic_forest = RandomForestClassifier(n_estimators=10, random_state=42)

basic_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(basic_forest, n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

In [8]:
# Train pipeline
basic_pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred = basic_pipeline.predict(X_test)

In [11]:
for i, col in enumerate(Y.columns):
    if i in [0, 9]: continue  # Skip bad column, TODO: Fix 0th column to not be 3 classes for no reason
    
    print("Column {}: {}".format(i, col))
    
    y_true = list(Y_test.values[:, i])
    y_pred = list(Y_pred[:, i])
    target_names = ['is_{}'.format(col), 'is_not_{}'.format(col)]
    print(classification_report(y_true, y_pred, target_names=target_names))

Column 1: request
                precision    recall  f1-score   support

    is_request       0.89      0.98      0.93      5449
is_not_request       0.80      0.43      0.56      1105

     micro avg       0.89      0.89      0.89      6554
     macro avg       0.85      0.70      0.75      6554
  weighted avg       0.88      0.89      0.87      6554

Column 2: offer
              precision    recall  f1-score   support

    is_offer       1.00      1.00      1.00      6524
is_not_offer       0.00      0.00      0.00        30

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Column 3: aid_related
                    precision    recall  f1-score   support

    is_aid_related       0.75      0.84      0.80      3880
is_not_aid_related       0.73      0.60      0.66      2674

         micro avg       0.74      0.74      0.74      6554
         macro avg       0.74      0

### 6. Improve your model
Use grid search to find better parameters. 

<div "I got this implementation from https://github.com/wenshihao1993/Udacity-DSND/blob/master/Data%20Engineering/Disaster%20Response%20Pipeline%20Project/models/train_classifier.py">
</div>

In [29]:
# Define pipeline
cv_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
])

# Define Parameters
parameters = {'clf__estimator__estimator__n_estimators': [10, 20, 50, 100, 200],
              'clf__estimator__estimator__criterion': ['gini', 'entropy'],
              'clf__estimator__estimator__max_depth': [4,5,6,7,8],
              'clf__estimator__estimator__max_features': ['auto', 'sqrt', 'log2']
              }

cv_pipeline = GridSearchCV(cv_pipeline, param_grid=parameters)

In [30]:
cv_pipeline.fit(X_train, Y_train)

C:\Users\sthornewillvonessen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.